In [ ]:
import pandas as pd
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
model  = 'gemma3:27b' # 'mistral-small3.1' #'gemma3:27b'
def promptModel(prompt):
    class Guess(BaseModel):
        guess: str
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.guess
print(promptModel('What is 1+2*3+5?  Return a single number.')) # force the model to load

In [ ]:
#
# Gemini Example
#
# '''You are an expert crossword puzzle solver. Your task is to provide the most likely single answer to a given crossword clue, constrained by the number of letters required. Your response MUST be a JSON object with with a single string.
#
#**Clue:** [Insert Crossword Clue Here]
#**Number of Letters:** [Insert Number of Letters Here]''',
#
def promptModel(prompt):
    class Guess(BaseModel):
        clue:       str
        length:     int
        answer:     str
        confidence: float
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.answer, guess.confidence
_prompts_ = [
'''You are a crossword-solving assistant. Given a crossword clue and the number of letters in the answer, your task is to return the most likely answer or a list of possible answers. Your response must be formatted as a JSON object with the following structure:

{
  "clue": "<original clue>",
  "length": <number of letters>,
  "answer": "<answer>"
  "confidence": <confidence score from 0 to 1>
}

Be concise. Include only relevant answers. Prioritize common crossword solutions. Limit the list to the top 3 most probable answers.

Example input:

Clue: "Capital of Italy"
Length: 4

Expected JSON output:

{
  "clue": "Capital of Italy",
  "length": 4,
  "answer": "Rome",
  "confidence": 0.98
}

Now solve the following clue:

Clue: "[Insert Crossword Clue Here]"
Length: [Insert Number of Letters Here]'''
]
_tiles_ = []
_lu_    = {'clue': [], 'length': [], 'answer': [], 'confidence': []}
for _prompt_ in _prompts_:
    xwords_copy = copy.deepcopy(xwords)
    for cluenum, orientation in xwords.allClueNumbersAndOrientations():
        clue           = xwords_copy.clue(cluenum, orientation)
        num_of_letters = xwords.numberOfLetters(cluenum, orientation)
        _prompt_to_mod_ = _prompt_
        _prompt_to_mod_ = _prompt_to_mod_.replace('[Insert Crossword Clue Here]', clue)
        _prompt_to_mod_ = _prompt_to_mod_.replace('[Insert Number of Letters Here]', str(num_of_letters))
        _answer_, _confidence_ = promptModel(_prompt_to_mod_)
        _lu_['clue'].append(clue); _lu_['length'].append(num_of_letters); _lu_['answer'].append(_answer_); _lu_['confidence'].append(_confidence_)
        if len(_answer_) != num_of_letters and ' ' in _answer_: 
            print('+',end='')
            _answer_ = _answer_.replace(' ', '')
        if len(_answer_) == num_of_letters:
            xwords_copy.guess(cluenum, orientation, _answer_)
            print('.',end='')
        else:
            print('!',end='')
    print(f' | {xwords_copy.characterLevelAccuracy()}')
    _tiles_.append(xwords_copy.smallMultipleSVG())
_tiles_.append(rt.xy(pd.DataFrame(_lu_), x_field='confidence', y_field='confidence', dot_size=None, render_x_distribution=10, distribution_style='inside', w=128, h=128))
rt.table(_tiles_, per_row=10, spacer=10)

In [ ]:
_prompts_ = ['''You are a crossword-solving assistant. Given a crossword clue, the number of letters in the answer, and an optional letter pattern (using underscores _ for unknown letters), return the most likely answer. Your response must be formatted as a JSON object with the following structure:

{
  "clue": "<original clue>",
  "length": <number of letters>,
  "pattern": "<letter pattern or null>",
  "word": "<possible answer>",
  "confidence": <confidence score from 0 to 1>
}

Use common crossword vocabulary and emphasize accuracy.

Letter pattern rules:

- Use _ to represent unknown letters.
- All known letters must appear in their correct positions.
- The pattern must match the specified length.

Example input:

Clue: "Capital of Italy"
Length: 4
Pattern: "R__E"

{
  "clue": "Capital of Italy",
  "length": 4,
  "pattern": "R__E",
  "answer": "Rome",
  "confidence": 0.98
}

Now solve the following clue:

Clue: "{clue}"
Length: {length}
Pattern: "{pattern}"
''']
def makePattern(cluenum, orientation, xw):
    num_of_letters = xw.numberOfLetters(cluenum, orientation)
    _cell_ = xw.cluenum_to_cell[cluenum]
    xi, yi = _cell_.xi, _cell_.yi
    _pattern_ = ''
    for i in range(num_of_letters):
        if orientation == 'across':
            _other_, _found_ = xw.cells[yi][xi + i], False
            for _num_orient_ in _other_.__guesses__:
                if _num_orient_[1] == 'down':
                    _pattern_ += _other_.__guesses__[_num_orient_]
                    _found_ = True
            if not _found_: _pattern_ += '_'
        else:
            _other_, _found_ = xw.cells[yi + i][xi], False
            for _num_orient_ in _other_.__guesses__:
                if _num_orient_[1] == 'across':
                    _pattern_ += _other_.__guesses__[_num_orient_]
                    _found_ = True
            if not _found_: _pattern_ += '_'
    return _pattern_
def promptModel(prompt):
    class Guess(BaseModel):
        clue:       str
        length:     int
        pattern:    str
        answer:     str
        confidence: float
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.answer, guess.confidence
_tiles_ = []
_lu_    = {'clue': [], 'length': [], 'answer': [], 'confidence': []}
for _prompt_ in _prompts_:
    xwords_copy = copy.deepcopy(xwords)
    for cluenum, orientation in xwords.allClueNumbersAndOrientations():
        clue           = xwords_copy.clue(cluenum, orientation)
        num_of_letters = xwords.numberOfLetters(cluenum, orientation)
        _prompt_to_mod_ = _prompt_
        _prompt_to_mod_ = _prompt_to_mod_.replace('{clue}',   clue)
        _prompt_to_mod_ = _prompt_to_mod_.replace('{pattern}', makePattern(cluenum, orientation, xwords_copy))
        _prompt_to_mod_ = _prompt_to_mod_.replace('{length}', str(num_of_letters))
        _answer_, _confidence_ = promptModel(_prompt_to_mod_)
        _lu_['clue'].append(clue); _lu_['length'].append(num_of_letters); _lu_['answer'].append(_answer_); _lu_['confidence'].append(_confidence_)
        if len(_answer_) != num_of_letters and ' ' in _answer_: 
            print('+',end='')
            _answer_ = _answer_.replace(' ', '')
        if len(_answer_) == num_of_letters:
            xwords_copy.guess(cluenum, orientation, _answer_)
            print('.',end='')
        else:
            print('!',end='')
    print(f' | {xwords_copy.characterLevelAccuracy()}')
    _tiles_.append(xwords_copy.smallMultipleSVG())
_tiles_.append(rt.xy(pd.DataFrame(_lu_), x_field='confidence', y_field='confidence', dot_size=None, render_x_distribution=10, distribution_style='inside', w=128, h=128))
rt.table(_tiles_, per_row=10, spacer=10)

In [ ]:
import networkx as nx
g_in,  g_out  = xwords_copy.sweepClipGraphs()
def networkXToDataFrame(g):
    _lu_ = {'fm': [], 'to': []}
    for edge in g.edges:
        _lu_['fm'].append(str(edge[0]))
        _lu_['to'].append(str(edge[1]))
    return pd.DataFrame(_lu_)
df_in, df_out = networkXToDataFrame(g_in),  networkXToDataFrame(g_out)
_relates_     = [('fm','to')]
g_in,  g_out  = rt.createNetworkXGraph(df_in, _relates_), rt.createNetworkXGraph(df_out, _relates_)
pos_in, pos_out = nx.spring_layout(g_in),     nx.spring_layout(g_out)
params = {'relationships':_relates_, 'w':512, 'h':512, 'node_size':'small'}
rt.tile([rt.linkNode(df_in,  pos=pos_in,  **params), 
         rt.linkNode(df_out, pos=pos_out, **params)], spacer=10)

In [ ]:
_prompts_ = ['''You are a crossword-solving assistant. Given a crossword clue, the number of letters in the answer, and an optional letter pattern (using underscores _ for unknown letters), return the three most likely answers. Your response must be formatted as a JSON object with the following structure:

{
  "clue": "<original clue>",
  "length": <number of letters>,
  "pattern": "<letter pattern or null>",
  "answer1": "<possible answer>",
  "confidence1": <confidence score from 0 to 1>,
  "answer2": "<possible answer>",
  "confidence2": <confidence score from 0 to 1>,
  "answer3": "<possible answer>",
  "confidence3": <confidence score from 0 to 1>
}

Use common crossword vocabulary and emphasize accuracy.

Letter pattern rules:

- Use _ to represent unknown letters.
- All known letters must appear in their correct positions.
- The pattern must match the specified length.

Example input:

Clue: "Capital of Italy"
Length: 4
Pattern: "R__E"

{
  "clue": "Capital of Italy",
  "length": 4,
  "pattern": "R__E",
  "answer1": "Rome",
  "confidence1": 0.98,
  "answer2": "Rope",
  "confidence2": 0.5,
  "answer3": "Florence",
  "confidence3": 0.01,                          
}

Now solve the following clue:

Clue: "{clue}"
Length: {length}
Pattern: "{pattern}"
''']
def makePattern(cluenum, orientation, xw):
    num_of_letters = xw.numberOfLetters(cluenum, orientation)
    _cell_ = xw.cluenum_to_cell[cluenum]
    xi, yi = _cell_.xi, _cell_.yi
    _pattern_ = ''
    for i in range(num_of_letters):
        if orientation == 'across':
            _other_, _found_ = xw.cells[yi][xi + i], False
            for _num_orient_ in _other_.__guesses__:
                if _num_orient_[1] == 'down':
                    _pattern_ += _other_.__guesses__[_num_orient_]
                    _found_ = True
            if not _found_: _pattern_ += '_'
        else:
            _other_, _found_ = xw.cells[yi + i][xi], False
            for _num_orient_ in _other_.__guesses__:
                if _num_orient_[1] == 'across':
                    _pattern_ += _other_.__guesses__[_num_orient_]
                    _found_ = True
            if not _found_: _pattern_ += '_'
    return _pattern_
def promptModel(prompt, num_of_letters):
    class Guess(BaseModel):
        clue:       str
        length:     int
        pattern:    str
        answer1:     str
        confidence1: float
        answer2:     str
        confidence2: float
        answer3:     str
        confidence3: float
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    if   len(guess.answer1) == num_of_letters: return guess.answer1, guess.confidence1
    elif len(guess.answer2) == num_of_letters: return guess.answer2, guess.confidence2
    elif len(guess.answer3) == num_of_letters: return guess.answer3, guess.confidence3
    return guess.answer1, guess.confidence1
_tiles_ = []
_lu_    = {'clue': [], 'length': [], 'answer': [], 'confidence': []}
for _prompt_ in _prompts_:
    xwords_copy = copy.deepcopy(xwords)
    for cluenum, orientation in xwords.allClueNumbersAndOrientations():
        clue           = xwords_copy.clue(cluenum, orientation)
        num_of_letters = xwords.numberOfLetters(cluenum, orientation)
        _prompt_to_mod_ = _prompt_
        _prompt_to_mod_ = _prompt_to_mod_.replace('{clue}',   clue)
        _prompt_to_mod_ = _prompt_to_mod_.replace('{pattern}', makePattern(cluenum, orientation, xwords_copy))
        _prompt_to_mod_ = _prompt_to_mod_.replace('{length}', str(num_of_letters))
        _answer_, _confidence_ = promptModel(_prompt_to_mod_, num_of_letters)
        _lu_['clue'].append(clue); _lu_['length'].append(num_of_letters); _lu_['answer'].append(_answer_); _lu_['confidence'].append(_confidence_)
        if len(_answer_) != num_of_letters and ' ' in _answer_: 
            print('+',end='')
            _answer_ = _answer_.replace(' ', '')
        if len(_answer_) == num_of_letters:
            xwords_copy.guess(cluenum, orientation, _answer_)
            print('.',end='')
        else:
            print('!',end='')
    print(f' | {xwords_copy.characterLevelAccuracy()}')
    _tiles_.append(xwords_copy.smallMultipleSVG())
_tiles_.append(rt.xy(pd.DataFrame(_lu_), x_field='confidence', y_field='confidence', dot_size=None, render_x_distribution=10, distribution_style='inside', w=128, h=128))
rt.table(_tiles_, per_row=10, spacer=10)